## Imports

In [1]:
import sys
import os
import random
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
import h5py
from h5py import File as HDF5File

import enum
print(enum.__file__)

/usatlas/atlas01/atlasdisk/users/atlas_wifeng/sw/miniconda/envs/top-tagging/lib/python3.10/enum.py


In [3]:
# Oh boy this might take a while
import tensorflow as tf
from tensorflow import keras

from keras.layers import Lambda, Input
from keras.layers import Dropout, Flatten, Dense
import keras.backend as K
from keras.models import Sequential, Model 
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import concatenate

2023-08-23 04:37:50.674766: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-23 04:37:53.453528: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
import yaml
with open("../config.yaml") as fin:
    config = yaml.safe_load(fin)
    data_dir = config["data_dir"]
    model_dir = config["model_dir"]
    output_dir = config["output_dir"]

In [5]:
task_name = "axion2"

## Loading data

See `cnn/data.py`.

In [6]:
from cnn_data import load_data
X_train, Y_train, X_test, Y_test = load_data(task_name)

load_data from datafile /usatlas/atlas01/atlasdisk/users/atlas_wifeng/photon-jet/data/h5/pi0_40-250GeV_100k.h5
load_data from datafile /usatlas/atlas01/atlasdisk/users/atlas_wifeng/photon-jet/data/h5/gamma_40-250GeV_100k.h5
load_data from datafile /usatlas/atlas01/atlasdisk/users/atlas_wifeng/photon-jet/data/h5/axion2_40-250GeV_100k.h5


In [7]:
print("Training shapes:")
print("  inputs:", X_train[0].shape, X_train[1].shape)
print("  labels:", Y_train.shape)

Training shapes:
  inputs: (210000, 4, 16, 1) (210000, 4, 128, 1)
  labels: (210000,)


## Build the model

In [8]:
visible1 = Input(shape=(4, 16, 1))
conv11 = Conv2D(32, kernel_size=4, activation='relu', padding='same')(visible1)
pool11 = MaxPooling2D(pool_size=(2, 2))(conv11)
conv12 = Conv2D(16, kernel_size=4, activation='relu', padding='same')(pool11)
pool12 = MaxPooling2D(pool_size=(2, 2))(conv12)
flat1 = Flatten()(pool12)
print ('cov11.shape', conv11.shape)
print ('pool1.shape', pool11.shape)
print ('cov12.shape', conv12.shape)
print ('pool2.shape', pool12.shape)

cov11.shape (None, 4, 16, 32)
pool1.shape (None, 2, 8, 32)
cov12.shape (None, 2, 8, 16)
pool2.shape (None, 1, 4, 16)


2023-08-23 04:38:04.567403: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15363 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:81:00.0, compute capability: 6.0


In [9]:
visible2 = Input(shape=(4, 128, 1))
conv21 = Conv2D(32, kernel_size=4, activation='relu', padding='same')(visible2)
pool21 = MaxPooling2D(pool_size=(2, 2))(conv21)
conv22 = Conv2D(16, kernel_size=4, activation='relu', padding='same')(pool21)
pool22 = MaxPooling2D(pool_size=(2, 2))(conv22)
flat2 = Flatten()(pool22)
print ('cov21.shape', conv21.shape)
print ('poo21.shape', pool21.shape)
print ('cov22.shape', conv22.shape)
print ('poo22.shape', pool22.shape)

cov21.shape (None, 4, 128, 32)
poo21.shape (None, 2, 64, 32)
cov22.shape (None, 2, 64, 16)
poo22.shape (None, 1, 32, 16)


In [10]:
merge = concatenate([flat1, flat2])

In [11]:
# interpretation model
hidden1 = Dense(10, activation='relu')(merge)
hidden2 = Dense(10, activation='relu')(hidden1)
output = Dense(3, activation='sigmoid')(hidden2)
cnn = Model(inputs=[visible1, visible2], outputs=output)

cnn.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [12]:
# summarize layers
print(cnn.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 4, 16, 1)]   0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 4, 128, 1)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 4, 16, 32)    544         ['input_1[0][0]']                
                                                                                                  
 conv2d_2 (Conv2D)              (None, 4, 128, 32)   544         ['input_2[0][0]']                
                                                                                              

## Train the model

In [ ]:
# Takes about 10 min to train 50 epochs. Reaches >99% accuracy!
history = cnn.fit(
    X_train, Y_train,
    epochs=50, batch_size=100,
    validation_data=(X_test, Y_test),
)

In [ ]:
# Save this model
os.makedirs(f"{model_dir}/cnn", exist_ok=True)

print(f"task name: {task_name}")
cnn.save(f"{model_dir}/cnn/{task_name}_cnn")

In [ ]:
import json

os.makedirs(f"{output_dir}/cnn", exist_ok=True)
with open(f"{output_dir}/cnn/{task_name}_train_history.json", "w") as fout:
    json.dump({
        key: history.history[key] for key in ["loss", "val_loss", "acc", "val_acc"]
    }, fout)